In [1]:
# -*- coding: utf-8 -*-
# author：Cookly
from util import DataLoader, Features
from SBBTree_ONLINE import SBBTree
import pandas as pd 
import numpy as np 
from datetime import datetime, timedelta
import lightgbm as lgb

In [2]:
# test code
Data = DataLoader(
				FILE_jdata_sku_basic_info='../data/jdata_sku_basic_info.csv',
				FILE_jdata_user_action='../data/jdata_user_action.csv',
				FILE_jdata_user_basic_info='../data/jdata_user_basic_info.csv',
				FILE_jdata_user_comment_score='../data/jdata_user_comment_score.csv',
				FILE_jdata_user_order='../data/jdata_user_order.csv'
			)
# train data
TrainFeatures = Features(
						DataLoader=Data,
						PredMonthBegin = datetime(2017, 4, 1),
						PredMonthEnd = datetime(2017, 4, 30),
						FeatureMonthList = [(datetime(2017, 3, 1), datetime(2017, 3, 31), 1),\
									(datetime(2017, 1, 1), datetime(2017, 3, 31), 3),\
									(datetime(2016, 10, 1), datetime(2017, 3, 31), 6)],
						MakeLabel = True
					)
# pred data
PredFeatures = Features(
					DataLoader=Data,
					PredMonthBegin = datetime(2017, 5, 1),
					PredMonthEnd = datetime(2017, 5, 31),
					FeatureMonthList = [(datetime(2017, 4, 1), datetime(2017, 4, 30), 1),\
									(datetime(2017, 2, 1), datetime(2017, 4, 30), 3),\
									(datetime(2016, 11, 1), datetime(2017, 4, 30), 6)],
					MakeLabel = False
				)

In [5]:
train_features = TrainFeatures.TrainColumns

In [15]:
cols = TrainFeatures.IDColumns + TrainFeatures.LabelColumns + train_features
							
###########
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
###############################################################
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)

In [11]:
# train 下个月购买次数预测 回归模型
train_features = TrainFeatures.TrainColumns
train_label_BuyNum = 'Label_30_101_BuyNum'

X = TrainFeatures.data_BuyOrNot_FirstTime[train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum].values

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_BuyNum] = model.predict(X_pred)

C:\Program Files\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[1]	valid_0's auc: 0.802053	valid_0's l2: 0.749531
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.803202	valid_0's l2: 0.734705
[3]	valid_0's auc: 0.80378	valid_0's l2: 0.721143
[4]	valid_0's auc: 0.804944	valid_0's l2: 0.706859
[5]	valid_0's auc: 0.80538	valid_0's l2: 0.695459
[6]	valid_0's auc: 0.806379	valid_0's l2: 0.685605
[7]	valid_0's auc: 0.806165	valid_0's l2: 0.677238
[8]	valid_0's auc: 0.806286	valid_0's l2: 0.680476
[9]	valid_0's auc: 0.806819	valid_0's l2: 0.671295
[10]	valid_0's auc: 0.806712	valid_0's l2: 0.663361
[11]	valid_0's auc: 0.807223	valid_0's l2: 0.656506
[12]	valid_0's auc: 0.80695	valid_0's l2: 0.658978
[13]	valid_0's auc: 0.807585	valid_0's l2: 0.652342
[14]	valid_0's auc: 0.808124	valid_0's l2: 0.646614
[15]	valid_0's auc: 0.808489	valid_0's l2: 0.642406
[16]	valid_0's auc: 0.808637	valid_0's l2: 0.638514
[17]	valid_0's auc: 0.808473	valid_0's l2: 0.634383
[18]	valid_0's auc: 0.808762	valid_0's l2: 0.631376
[19]	valid_0'

In [12]:
###############################################################
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
model = SBBTree(params=params, stacking_num=5, bagging_num=3,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# train 当月首次购买时间预测 回归模型
train_label_FirstTime = 'Label_30_101_FirstTime'
X = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_features].values 
y = TrainFeatures.data_BuyOrNot_FirstTime[TrainFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime]>0][train_label_FirstTime].values 

X_pred = PredFeatures.data_BuyOrNot_FirstTime[train_features].values 

model.fit(X,y)
PredFeatures.data_BuyOrNot_FirstTime[train_label_FirstTime] = model.predict(X_pred)

[1]	valid_0's l2: 60.8466
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 60.5828
[3]	valid_0's l2: 60.3303
[4]	valid_0's l2: 60.1049
[5]	valid_0's l2: 59.9017
[6]	valid_0's l2: 59.7214
[7]	valid_0's l2: 59.5546
[8]	valid_0's l2: 59.6222
[9]	valid_0's l2: 59.4662
[10]	valid_0's l2: 59.3227
[11]	valid_0's l2: 59.1782
[12]	valid_0's l2: 59.2315
[13]	valid_0's l2: 59.1088
[14]	valid_0's l2: 58.9777
[15]	valid_0's l2: 58.8621
[16]	valid_0's l2: 58.7631
[17]	valid_0's l2: 58.6739
[18]	valid_0's l2: 58.5914
[19]	valid_0's l2: 58.5248
[20]	valid_0's l2: 58.4571
[21]	valid_0's l2: 58.4792
[22]	valid_0's l2: 58.4018
[23]	valid_0's l2: 58.337
[24]	valid_0's l2: 58.2758
[25]	valid_0's l2: 58.2119
[26]	valid_0's l2: 58.1727
[27]	valid_0's l2: 58.1402
[28]	valid_0's l2: 58.1669
[29]	valid_0's l2: 58.1211
[30]	valid_0's l2: 58.0846
[31]	valid_0's l2: 58.0886
[32]	valid_0's l2: 58.0498
[33]	valid_0's l2: 58.0156
[34]	valid_0's l2: 57.9789
[35]	valid_0's l2: 58.0053
[3

In [14]:
# submit
columns = ['user_id'] + [train_label_BuyNum] + [train_label_FirstTime]
out_submit = PredFeatures.data_BuyOrNot_FirstTime[columns].sort_values(['Label_30_101_BuyNum'],ascending=False)
out_submit[train_label_FirstTime] = out_submit[train_label_FirstTime].map(lambda day: datetime(2017, 5, 1)+timedelta(days=int(day+0.49-1)))

out_submit = out_submit[['user_id']+[train_label_FirstTime]]
out_submit.columns = ['user_id','pred_date']
out_submit.head(50000).to_csv('../submit/predict2.csv',index=False,header=True)